In [17]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [18]:
cols = ['ID(original)', 'ID' , 'Q' , 'A']
data = pd.read_csv('COQB-19_crowdsourced_06202020.csv', sep=',', names=cols, header=None, encoding='latin-1', skiprows=1) # TODO: add skiprows
# TODO: drop rows with na
data = data.dropna()
ID= data['ID'].tolist()
corpus = data['Q'].tolist()
Answers = data['A'].tolist()

In [19]:
pairs = list(zip(corpus,Answers))

In [20]:
max_answer_words = 50

In [21]:
input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
# TODO: use all pairs
for line in pairs[:]:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc)[:max_answer_words])
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

In [22]:
batch_size = 256

In [23]:
def encode_question_answer(input_doc, target_doc):
  """ Encode the question andanswer """
  encoder_input_data = np.zeros(
    (max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
  decoder_input_data = np.zeros(
     (max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
  decoder_target_data = np.zeros(
    (max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
  
  for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
    #Assign 1. for the current line, timestep, & word in encoder_input_data
    encoder_input_data[timestep, input_features_dict[token]] = 1
    
  for timestep, token in enumerate(target_doc.split()):
    decoder_input_data[timestep, target_features_dict[token]] = 1
    if timestep > 0:
      decoder_target_data[timestep - 1, target_features_dict[token]] = 1  
  # TODO: we  don't really need the second output, but for some reason I have found the dataset will not batch properly without it.
  return (encoder_input_data,  decoder_input_data),  (decoder_target_data,decoder_target_data)

In [24]:
def generator_question_answer():
  """ Encode the question andanswer """
  idx = 0
  while idx < len(pairs):
    yield encode_question_answer(input_docs[idx], target_docs[idx])
    idx +=1

In [25]:
dataset = tf.data.Dataset.from_generator(generator_question_answer,
                                          output_types=((tf.float32, tf.float32), (tf.float32, tf.float32)),
                                          output_shapes = (([max_encoder_seq_length, num_encoder_tokens],[max_decoder_seq_length,num_decoder_tokens]),
                                          ([max_decoder_seq_length,num_decoder_tokens],[max_decoder_seq_length,num_decoder_tokens])),)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [26]:
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
epochs = 600
#Encoder
encoder_inputs = Input(shape= (max_encoder_seq_length, num_encoder_tokens), dtype=tf.float32, name='enc_input')
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(max_decoder_seq_length, num_decoder_tokens), dtype=tf.float32, name='dec_input')
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [27]:
#Model
# TODO: we  don't really need the second output, but for some reason I have found the dataset will not batch properlywithout it.
# We can adjust for its presence by dividing the learning rate by 2.
training_model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[decoder_outputs,decoder_outputs])
#Compiling
training_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003), loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training

In [28]:
training_model.fit(dataset, epochs = epochs)

Epoch 1/600
10/10 [==============================] - 25s 243ms/step - loss: 11.0251 - dense_1_loss: 5.5126 - dense_1_1_loss: 5.5126 - dense_1_accuracy: 0.0403 - dense_1_1_accuracy: 0.0403
Epoch 2/600
10/10 [==============================] - 3s 261ms/step - loss: 9.2910 - dense_1_loss: 4.6455 - dense_1_1_loss: 4.6455 - dense_1_accuracy: 0.0415 - dense_1_1_accuracy: 0.0415
Epoch 3/600
10/10 [==============================] - 3s 256ms/step - loss: 9.1734 - dense_1_loss: 4.5867 - dense_1_1_loss: 4.5867 - dense_1_accuracy: 0.0416 - dense_1_1_accuracy: 0.0416
Epoch 4/600
10/10 [==============================] - 3s 268ms/step - loss: 9.0904 - dense_1_loss: 4.5452 - dense_1_1_loss: 4.5452 - dense_1_accuracy: 0.0415 - dense_1_1_accuracy: 0.0415
Epoch 5/600
10/10 [==============================] - 3s 259ms/step - loss: 9.0458 - dense_1_loss: 4.5229 - dense_1_1_loss: 4.5229 - dense_1_accuracy: 0.0433 - dense_1_1_accuracy: 0.0433
Epoch 6/600
10/10 [==============================] - 3s 260ms/step -

In [29]:
training_model.save('training_model.h5')

In [30]:
from keras.models import load_model
training_model = load_model('training_model.h5')

In [31]:
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

In [32]:
latent_dim = 256
decoder_infer_inputs = Input(shape=(1, num_decoder_tokens), dtype=tf.float32, name='dec_infer_input')
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_infer_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_infer_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [33]:
def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    word_idx = 0
    stop_condition = False
    while not stop_condition:
        #Predicting output tokens with probabilities and states
        output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]
        decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
        if (sampled_token == '<END>' or word_idx >= max_decoder_seq_length-1):
          stop_condition = True
        else:
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          word_idx+=1
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [34]:
def string_to_matrix(user_input):
  tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
  user_input_matrix = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
  for timestep, token in enumerate(tokens):
    if token in input_features_dict:
      user_input_matrix[0, timestep, input_features_dict[token]] = 1.
  return user_input_matrix

#Method that will create a response using seq2seq model we built
def generate_response(user_input):
  input_matrix = string_to_matrix(user_input)
  chatbot_response = decode_response(input_matrix)
  #Remove <START> and <END> tokens from chatbot_response
  chatbot_response = chatbot_response.replace("<START>",'')
  chatbot_response = chatbot_response.replace("<END>",'')
  return chatbot_response

In [35]:
import os
if not os.path.exists('templates'):
  os.makedirs('templates')
if not os.path.exists('static'):
  os.makedirs('static')


In [36]:
home= '''

<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <title>InferMedica Chatbot</title>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta http-equiv="X-UA-Compatible" content="ie=edge">
  <link rel="stylesheet" href="/static/style.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.2.1/jquery.min.js"></script>
</head>

<body>
  <!-- partial:index.partial.html -->
  <section class="msger">
    <header class="msger-header">
      <div class="msger-header-title">
        <i></i> InferMedica Chatbot <i></i>
      </div>
    </header>

    <main class="msger-chat">
      <div class="msg left-msg">
        <div class="msg-img" style="background-image: url(https://image.flaticon.com/icons/svg/327/327779.svg)"></div>

        <div class="msg-bubble">
          <div class="msg-info">
            <div class="msg-info-name">InferMedica</div>
        
          </div>

          <div class="msg-text">
            Hi, welcome to InferMedica ChatBot! How can I help you?! 😄
          </div>
        </div>
      </div>

    </main>

    <form class="msger-inputarea">
      <input type="text" class="msger-input" id="textInput" placeholder="Type your message...">
      <button type="submit" class="msger-send-btn">Send</button>
    </form>
  </section>
  <!-- partial -->
  <script src='https://use.fontawesome.com/releases/v5.0.13/js/all.js'></script>
  <script>

    const msgerForm = get(".msger-inputarea");
    const msgerInput = get(".msger-input");
    const msgerChat = get(".msger-chat");


    // Icons made by Freepik from www.flaticon.com
    const BOT_IMG = "https://image.flaticon.com/icons/svg/327/327779.svg";
    const PERSON_IMG = "https://image.flaticon.com/icons/svg/145/145867.svg";
    const BOT_NAME = "ChatBot";
    const PERSON_NAME = "You";

    msgerForm.addEventListener("submit", event => {
      event.preventDefault();

      const msgText = msgerInput.value;
      if (!msgText) return;

      appendMessage(PERSON_NAME, PERSON_IMG, "right", msgText);
      msgerInput.value = "";
      botResponse(msgText);
    });

    function appendMessage(name, img, side, text) {
      //   Simple solution for small apps
      const msgHTML = `
<div class="msg ${side}-msg">
  <div class="msg-img" style="background-image: url(${img})"></div>

  <div class="msg-bubble">
    <div class="msg-info">
      <div class="msg-info-name">${name}</div>
      <div class="msg-info-time">${formatDate(new Date())}</div>
    </div>

    <div class="msg-text">${text}</div>
  </div>
</div>
`;

      msgerChat.insertAdjacentHTML("beforeend", msgHTML);
      msgerChat.scrollTop += 500;
    }

    function botResponse(rawText) {

      // Bot Response
      $.get("/get", { msg: rawText }).done(function (data) {
        console.log(rawText);
        console.log(data);
        const msgText = data;
        appendMessage(BOT_NAME, BOT_IMG, "left", msgText);

      });

    }


    // Utils
    function get(selector, root = document) {
      return root.querySelector(selector);
    }

    function formatDate(date) {
      const h = "0" + date.getHours();
      const m = "0" + date.getMinutes();

      return `${h.slice(-2)}:${m.slice(-2)}`;
    }



  </script>

</body>

</html>
'''


file=open("templates/home.html", "w")
file.write(home)
file.close()


In [37]:
!pip install flask-ngrok
!pip install flask==0.12.2


     |████████████████████████████████| 92kB 5.7MB/s 
  Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("home.html")
@app.route("/get")
def get_bot_response():
    user_input = request.args.get('msg')
    return generate_response(user_input)
app.run()


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://12e1e47ac006.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [17/Jul/2021 10:40:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:40:05] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [17/Jul/2021 10:40:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:40:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Jul/2021 10:40:15] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [17/Jul/2021 10:41:16] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:41:17] "GET /static/medical.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:41:26] "GET /get?msg=hi HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:44:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:44:03] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:44:03] "GET /static/medical.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:44:05] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Jul/2021 10:45:11] "GET /get?msg=hello HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:47:30] "GET /get?msg=what%20is%20covid%2019%3